# Dataset pre processing

This notbook focuses on the bncc dataset pre processing aiming the training of a classification model for "etapa do conhecimento" prediction

In [24]:
import pandas as pd
import numpy as np
import html
import nltk
import regex as re
import string
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

## Selecting and filtering the features that will be used in the model

In [3]:
# Importing Data

df_bncc = pd.read_csv('/home/pdsmoreau/data-science/tera/bncc-classifier/data/raw/bncc_first_classifier.csv')

In [4]:
# Making a copy of the dataset and visualizing it

df_bncc_copy = df_bncc.copy()

df_bncc_copy.head()

,id,question,bulletType,name,slug,name.1,slug.1,name.2
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,historia-da-arte,Arte,arte,Fundamental II
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,algebra-equacoes-do-2o-grau-equacao-do-2o-grau...,Matemática,matematica,Fundamental II
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,4,Outros,outros,Espanhol,espanhol,Fundamental II
3,106855,<p><div> A respeito dos serviços públicos e da...,2,Responsabilidade Civil do Estado,responsabilidade-civil-do-estado,Direito Administrativo,direito-administrativo,Concurso
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,outros,Química,quimica,Fundamental II


In [5]:
# Dropping unuseful columns

df_bncc_copy = df_bncc_copy[['id','question', 'name.1']]

df_bncc_copy.head()

,id,question,name.1
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,Espanhol
3,106855,<p><div> A respeito dos serviços públicos e da...,Direito Administrativo
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química


In [6]:
# Renaming dataset columns

df_bncc_copy.columns = ['id', 'questions', 'target']

df_bncc_copy.head(3)

,id,questions,target
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,Espanhol


In [7]:
# Observing the classes of the target to drop, aiming to comprehend only the topics of bncc

df_bncc_copy['target'].value_counts()

Matemática                                                                                                 12539
Língua Portuguesa                                                                                          11887
Geografia                                                                                                   9266
História                                                                                                    9072
Inglês                                                                                                      8103
Ciências                                                                                                    7099
Física                                                                                                      5537
Filosofia                                                                                                   3728
Arte                                                                                            

In [8]:
df_bncc_copy.shape

(100000, 3)

In [9]:
# Selecting observations which are the target are included on BNCC

df_bncc_copy_targets_bncc = df_bncc_copy.loc[df_bncc_copy['target'].isin(['Língua Portuguesa', 'Arte', 'Educação Física',
                                                                    'Inglês', 'Matemática', 'Ciências', 'Geografia',
                                                                    'História', 'Ensino Religioso', 'Biologia', 'Física', 'Química'])]


Obs: in the setp of making the final solution, we must observe if the Topic of knowledge fits with the Setp of knowledge predicted.
For exemple: if the first code was EM, than, the topic must be Ciências Humanas
else EF , than the topic must be Geografia (or other in Cências Humanas)

In [10]:
# Observing if the filtering was correctly applied

df_bncc_copy_targets_bncc

,id,questions,target
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte
...,...,...,...
99994,2447693,<p>Leia a reportagem e responda a quest&atilde...,Língua Portuguesa
99995,4330761,"<p>Para responder à questão, leia o trecho do ...",Língua Portuguesa
99996,3249373,"<h1 style=""list-style:none;margin:0px 0px 20px...",Educação Física
99998,4150214,<p>Os machos e fêmeas de mosquitos do gênero <...,Ciências


In [11]:
# Observing if the filtering was correctly applied by visualizing the clases

df_bncc_copy_targets_bncc['target'].value_counts()


Matemática           12539
Língua Portuguesa    11887
Geografia             9266
História              9072
Inglês                8103
Ciências              7099
Física                5537
Arte                  3681
Química               3276
Biologia              3133
Educação Física       2386
Ensino Religioso       468
Name: target, dtype: int64

In [12]:
# How many observations do we have now?

df_bncc_copy_targets_bncc.shape

(76447, 3)

In [13]:
# Encoding the target with labels for the classifier

df_bncc_copy_targets_bncc['target_labels'] = df_bncc_copy_targets_bncc['target'].replace(
    {'Matemática': 0, 'Língua Portuguesa': 1,
     'Geografia': 2, 'História': 3,
     'Inglês': 4, 'Ciências': 5,
     'Física': 6, 'Arte': 7,
     'Química': 8, 'Biologia': 9,
     'Educação Física': 10, 'Ensino Religioso': 11})

df_bncc_copy_targets_bncc


/tmp/ipykernel_33693/3514497660.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc['target_labels'] = df_bncc_copy_targets_bncc['target'].replace(


,id,questions,target,target_labels
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte,7
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática,0
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química,8
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História,3
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte,7
...,...,...,...,...
99994,2447693,<p>Leia a reportagem e responda a quest&atilde...,Língua Portuguesa,1
99995,4330761,"<p>Para responder à questão, leia o trecho do ...",Língua Portuguesa,1
99996,3249373,"<h1 style=""list-style:none;margin:0px 0px 20px...",Educação Física,10
99998,4150214,<p>Os machos e fêmeas de mosquitos do gênero <...,Ciências,5


## Cleaning the dataset

In [14]:
# Transforming all of questions to string

df_bncc_copy_targets_bncc['questions'] = df_bncc_copy_targets_bncc['questions'].astype(str)


/tmp/ipykernel_33693/1033037570.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc['questions'] = df_bncc_copy_targets_bncc['questions'].astype(str)


In [15]:
# Removing html tags from questions

df_bncc_copy_targets_bncc['questions_tags_free'] = df_bncc_copy_targets_bncc['questions'].apply(html.unescape)

df_bncc_copy_targets_bncc.head()

/tmp/ipykernel_33693/392354530.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc['questions_tags_free'] = df_bncc_copy_targets_bncc['questions'].apply(html.unescape)


,id,questions,target,target_labels,questions_tags_free
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte,7,<p>Com base em seus conhecimentos sobre o comp...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática,0,"<p>“Achar um número que, somado com 2 seja igu..."
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química,8,"<p style=""padding:0px;margin:10px 0px 0px;outl..."
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História,3,<p>Pagina nr. 54 do Livro didático.</p>
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte,7,<p>O que é CHIAROSCURO?</p>


In [16]:
# Removing HTML

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

df_bncc_copy_targets_bncc['questions_html_free'] = df_bncc_copy_targets_bncc['questions_tags_free'].apply(
    lambda text: remove_html(text))
df_bncc_copy_targets_bncc.head()

/tmp/ipykernel_33693/553084964.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc['questions_html_free'] = df_bncc_copy_targets_bncc['questions_tags_free'].apply(


,id,questions,target,target_labels,questions_tags_free,questions_html_free
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte,7,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o composi...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática,0,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual ..."
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química,8,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela pol...
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História,3,<p>Pagina nr. 54 do Livro didático.</p>,Pagina nr. 54 do Livro didático.
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte,7,<p>O que é CHIAROSCURO?</p>,O que é CHIAROSCURO?


In [17]:
# Lower Casing

df_bncc_copy_targets_bncc['questions_lower'] = df_bncc_copy_targets_bncc['questions_html_free'].str.lower()

df_bncc_copy_targets_bncc.head()

/tmp/ipykernel_33693/1171570181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc['questions_lower'] = df_bncc_copy_targets_bncc['questions_html_free'].str.lower()


,id,questions,target,target_labels,questions_tags_free,questions_html_free,questions_lower
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte,7,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática,0,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual ...","“achar um número que, somado com 2 seja igual ..."
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química,8,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História,3,<p>Pagina nr. 54 do Livro didático.</p>,Pagina nr. 54 do Livro didático.,pagina nr. 54 do livro didático.
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte,7,<p>O que é CHIAROSCURO?</p>,O que é CHIAROSCURO?,o que é chiaroscuro?


In [18]:
# Removing punctuation of the questions

PUNCT_TO_REMOVE = string.punctuation


def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))


df_bncc_copy_targets_bncc['questions_punctuation_free'] = df_bncc_copy_targets_bncc['questions_lower'].apply(
    lambda text: remove_punctuation(text))
df_bncc_copy_targets_bncc.head()


/tmp/ipykernel_33693/1298338587.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc['questions_punctuation_free'] = df_bncc_copy_targets_bncc['questions_lower'].apply(


,id,questions,target,target_labels,questions_tags_free,questions_html_free,questions_lower,questions_punctuation_free
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte,7,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática,0,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual ...","“achar um número que, somado com 2 seja igual ...",“achar um número que somado com 2 seja igual a...
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química,8,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História,3,<p>Pagina nr. 54 do Livro didático.</p>,Pagina nr. 54 do Livro didático.,pagina nr. 54 do livro didático.,pagina nr 54 do livro didático
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte,7,<p>O que é CHIAROSCURO?</p>,O que é CHIAROSCURO?,o que é chiaroscuro?,o que é chiaroscuro


In [19]:
# Removing Italic Quotes

# Função para remover aspas em itálico
def remove_italic_dquotes(text: str) -> str:
    pattern = re.compile(r'"')
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover aspas de abertura
def remove_open_dquotes(text: str) -> str:
    pattern = re.compile(r'“')
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover aspas de abertura
def remove_end_dquotes(text: str) -> str:
    pattern = re.compile(r'”')
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover aspas de abertura
def remove_italic_quotes(text: str) -> str:
    pattern = re.compile(r"'")
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover aspas de abertura
def remove_open_quotes(text: str) -> str:
    pattern = re.compile(r'‘')
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover aspas de abertura
def remove_end_quotes(text: str) -> str:
    pattern = re.compile(r'’')
    clean_text = re.sub(pattern, " ", text)
    return clean_text

# Função para remover aspas de abertura
def remove_quote(text: str) -> str:
    pattern = re.compile(r'‛')
    clean_text = re.sub(pattern, " ", text)
    return clean_text

In [20]:
df_bncc_copy_targets_bncc['questions_no_italic_quotes'] = df_bncc_copy_targets_bncc['questions_punctuation_free'].apply(remove_italic_quotes)
df_bncc_copy_targets_bncc['questions_no_open_quotes'] = df_bncc_copy_targets_bncc['questions_no_italic_quotes'].apply(remove_open_quotes)
df_bncc_copy_targets_bncc['questions_no_end_quotes'] = df_bncc_copy_targets_bncc['questions_no_open_quotes'].apply(remove_end_quotes)

df_bncc_copy_targets_bncc['questions_no_italic_dquotes'] = df_bncc_copy_targets_bncc['questions_no_end_quotes'].apply(remove_italic_dquotes)
df_bncc_copy_targets_bncc['questions_no_open_dquotes'] = df_bncc_copy_targets_bncc['questions_no_italic_dquotes'].apply(remove_open_dquotes)
df_bncc_copy_targets_bncc['questions_no_end_dquotes'] = df_bncc_copy_targets_bncc['questions_no_open_dquotes'].apply(remove_end_dquotes)

df_bncc_copy_targets_bncc['questions_no_quotes'] = df_bncc_copy_targets_bncc['questions_no_end_dquotes'].apply(remove_quote)

/tmp/ipykernel_33693/303628689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc['questions_no_italic_quotes'] = df_bncc_copy_targets_bncc['questions_punctuation_free'].apply(remove_italic_quotes)
/tmp/ipykernel_33693/303628689.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc['questions_no_open_quotes'] = df_bncc_copy_targets_bncc['questions_no_italic_quotes'].apply(remove_open_quotes)
/tmp/ipykernel_33693/303628689.py:3: SettingWithCopyWarning: 
A va

In [21]:
df_bncc_copy_targets_bncc.head()

,id,questions,target,target_labels,questions_tags_free,questions_html_free,questions_lower,questions_punctuation_free,questions_no_italic_quotes,questions_no_open_quotes,questions_no_end_quotes,questions_no_italic_dquotes,questions_no_open_dquotes,questions_no_end_dquotes,questions_no_quotes
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte,7,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática,0,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual ...","“achar um número que, somado com 2 seja igual ...",“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química,8,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História,3,<p>Pagina nr. 54 do Livro didático.</p>,Pagina nr. 54 do Livro didático.,pagina nr. 54 do livro didático.,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte,7,<p>O que é CHIAROSCURO?</p>,O que é CHIAROSCURO?,o que é chiaroscuro?,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro


In [22]:
from nltk.corpus import stopwords
nltk.download('stopwords')


# Removing Portuguese Stopwords

pt_stopwords = set(stopwords.words('portuguese'))


def remove_pt_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in pt_stopwords])


df_bncc_copy_targets_bncc['questions_no_pt_stopwords'] = df_bncc_copy_targets_bncc['questions_no_quotes'].apply(
    lambda text: remove_pt_stopwords(text))

df_bncc_copy_targets_bncc.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pdsmoreau/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/tmp/ipykernel_33693/382244489.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc['questions_no_pt_stopwords'] = df_bncc_copy_targets_bncc['questions_no_quotes'].apply(


,id,questions,target,target_labels,questions_tags_free,questions_html_free,questions_lower,questions_punctuation_free,questions_no_italic_quotes,questions_no_open_quotes,questions_no_end_quotes,questions_no_italic_dquotes,questions_no_open_dquotes,questions_no_end_dquotes,questions_no_quotes,questions_no_pt_stopwords
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte,7,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,base conhecimentos sobre compositor john cage ...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática,0,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual ...","“achar um número que, somado com 2 seja igual ...",“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...,achar número somado 2 igual inverso equações r...
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química,8,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,chuva ácida fenômeno provocado poluição atmosf...
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História,3,<p>Pagina nr. 54 do Livro didático.</p>,Pagina nr. 54 do Livro didático.,pagina nr. 54 do livro didático.,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 livro didático
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte,7,<p>O que é CHIAROSCURO?</p>,O que é CHIAROSCURO?,o que é chiaroscuro?,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,chiaroscuro


In [23]:
# Removing English Stopwords

english_stopwords = set(stopwords.words('english'))


def remove_english_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in english_stopwords])


df_bncc_copy_targets_bncc['questions_no_english_stopwords'] = df_bncc_copy_targets_bncc['questions_no_pt_stopwords'].apply(
    lambda text: remove_english_stopwords(text))

df_bncc_copy_targets_bncc.head()


/tmp/ipykernel_33693/438173031.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc['questions_no_english_stopwords'] = df_bncc_copy_targets_bncc['questions_no_pt_stopwords'].apply(


,id,questions,target,target_labels,questions_tags_free,questions_html_free,questions_lower,questions_punctuation_free,questions_no_italic_quotes,questions_no_open_quotes,questions_no_end_quotes,questions_no_italic_dquotes,questions_no_open_dquotes,questions_no_end_dquotes,questions_no_quotes,questions_no_pt_stopwords,questions_no_english_stopwords
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte,7,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática,0,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual ...","“achar um número que, somado com 2 seja igual ...",“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...,achar número somado 2 igual inverso equações r...,achar número somado 2 igual inverso equações r...
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química,8,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História,3,<p>Pagina nr. 54 do Livro didático.</p>,Pagina nr. 54 do Livro didático.,pagina nr. 54 do livro didático.,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 livro didático,pagina nr 54 livro didático
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte,7,<p>O que é CHIAROSCURO?</p>,O que é CHIAROSCURO?,o que é chiaroscuro?,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,chiaroscuro,chiaroscuro


In [25]:
# Removing Frequent words

cnt = Counter()

for text in df_bncc_copy_targets_bncc['questions_no_english_stopwords'].values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common(10)


[('texto', 12063),
 ('sobre', 11968),
 ('alternativa', 10015),
 ('ser', 9981),
 ('assinale', 8717),
 ('–', 8515),
 ('acesso', 7514),
 ('correta', 6716),
 ('abaixo', 6714),
 ('leia', 6661)]

In [26]:
# Removing Frequent Words

cnt = Counter()

freqwords = set([w for (w, wc) in cnt.most_common(10)])


def remove_freqwords(text):
    return " ".join([word for word in str(text).split() if word not in freqwords])


df_bncc_copy_targets_bncc['questions_no_freq_words'] = df_bncc_copy_targets_bncc['questions_no_english_stopwords'].apply(
    lambda text: remove_freqwords(text))

df_bncc_copy_targets_bncc.head()


/tmp/ipykernel_33693/2136918353.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc['questions_no_freq_words'] = df_bncc_copy_targets_bncc['questions_no_english_stopwords'].apply(lambda text: remove_freqwords(text))


,id,questions,target,target_labels,questions_tags_free,questions_html_free,questions_lower,questions_punctuation_free,questions_no_italic_quotes,questions_no_open_quotes,questions_no_end_quotes,questions_no_italic_dquotes,questions_no_open_dquotes,questions_no_end_dquotes,questions_no_quotes,questions_no_pt_stopwords,questions_no_english_stopwords,questions_no_freq_words
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte,7,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática,0,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual ...","“achar um número que, somado com 2 seja igual ...",“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...,achar número somado 2 igual inverso equações r...,achar número somado 2 igual inverso equações r...,achar número somado 2 igual inverso equações r...
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química,8,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História,3,<p>Pagina nr. 54 do Livro didático.</p>,Pagina nr. 54 do Livro didático.,pagina nr. 54 do livro didático.,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 livro didático,pagina nr 54 livro didático,pagina nr 54 livro didático
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte,7,<p>O que é CHIAROSCURO?</p>,O que é CHIAROSCURO?,o que é chiaroscuro?,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,chiaroscuro,chiaroscuro,chiaroscuro


In [27]:
n_rare_words = 10
rarewords = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])


def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in rarewords])


df_bncc_copy_targets_bncc["questions_no_rare-words"] = df_bncc_copy_targets_bncc['questions_no_freq_words'].apply(
    lambda text: remove_rarewords(text))

df_bncc_copy_targets_bncc.head()


/tmp/ipykernel_33693/641187719.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc["questions_no_rare-words"] = df_bncc_copy_targets_bncc['questions_no_freq_words'].apply(


,id,questions,target,target_labels,questions_tags_free,questions_html_free,questions_lower,questions_punctuation_free,questions_no_italic_quotes,questions_no_open_quotes,questions_no_end_quotes,questions_no_italic_dquotes,questions_no_open_dquotes,questions_no_end_dquotes,questions_no_quotes,questions_no_pt_stopwords,questions_no_english_stopwords,questions_no_freq_words,questions_no_rare-words
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Arte,7,<p>Com base em seus conhecimentos sobre o comp...,Com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,com base em seus conhecimentos sobre o composi...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...,base conhecimentos sobre compositor john cage ...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Matemática,0,"<p>“Achar um número que, somado com 2 seja igu...","“Achar um número que, somado com 2 seja igual ...","“achar um número que, somado com 2 seja igual ...",“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,“achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...,achar um número que somado com 2 seja igual a...,achar número somado 2 igual inverso equações r...,achar número somado 2 igual inverso equações r...,achar número somado 2 igual inverso equações r...,achar número somado 2 igual inverso equações r...
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Química,8,"<p style=""padding:0px;margin:10px 0px 0px;outl...",A chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,a chuva ácida é um fenômeno provocado pela pol...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...,chuva ácida fenômeno provocado poluição atmosf...
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,História,3,<p>Pagina nr. 54 do Livro didático.</p>,Pagina nr. 54 do Livro didático.,pagina nr. 54 do livro didático.,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 do livro didático,pagina nr 54 livro didático,pagina nr 54 livro didático,pagina nr 54 livro didático,pagina nr 54 livro didático
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Arte,7,<p>O que é CHIAROSCURO?</p>,O que é CHIAROSCURO?,o que é chiaroscuro?,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,o que é chiaroscuro,chiaroscuro,chiaroscuro,chiaroscuro,chiaroscuro


### Exporting the dataset

In [34]:
df_bncc_model_two_curated = df_bncc_copy_targets_bncc[['id', 'target', 'target_labels', 'questions_no_rare-words']]

In [36]:
# Renaming columns

df_bncc_model_two_curated.columns = ['id', 'target', 'target_labels', 'questions_curated']

In [37]:
# Changing the order of columns

df_bncc_model_two_curated = df_bncc_model_two_curated[['id', 'questions_curated', 'target', 'target_labels']]

df_bncc_model_two_curated.head()

,id,questions_curated,target,target_labels
0,3486670,base conhecimentos sobre compositor john cage ...,Arte,7
1,3343927,achar número somado 2 igual inverso equações r...,Matemática,0
4,3691951,chuva ácida fenômeno provocado poluição atmosf...,Química,8
6,1744035,pagina nr 54 livro didático,História,3
7,2525602,chiaroscuro,Arte,7


In [38]:
# Exporting

df_bncc_model_two_curated.to_csv('/home/pdsmoreau/data-science/tera/bncc-classifier/data/curated/df_bncc_model_two_curated_pedro_02032022.csv')